# Использование DOM и Canvas для визуализации. WebGL.

## DOM vs Canvas

Источники: [1](https://drabstract.medium.com/your-guide-to-when-to-use-a-javascript-canvas-library-or-framework-efb30d526797)

С появлением HTML5 Javascript был разделен на 2 части: ядро языка и набор Web API. Два крупных Web API:
1. DOM
2. Canvas

Несмотря на то, что `canvas` - это HTML тэг и формально часть DOM, он рассматривается отдельно из-за деталей реализации.

Canvas отризовывает битовую карту (bitmap), а Canvas API позволяет делать визуализации динамичными за счет изменении этой битовой карты при помощи JS.

Canvas:
* позволяет двигать объекты в свободной форме (хорошо для создания изображений и игр);
* текст может быть добавлен на canvas и может быть сделан адаптивным, но не может быть выделен или найден при помощи поиска, как в случае DOM (но можно вынести текст в DOM);
* canvas является частью HTML5 и позволяет своим пользователям создавать динамические 2D-фигуры, визуализируемые скриптом
* интерактивность преобладает над навигацией.

DOM:
* удобно отображать информацию (особенно текстовую);
* текст преобладает над изображениями, аудио и видео; навигация преобладает над интерактивностью.

Пример интерактивной визуализации на canvas: https://dygraphs.com/

![DOM/Canvas](https://miro.medium.com/max/700/1*93xRM1Iqm5PT-t8PfFnU1w.png)

## WebGL

Источники: [1](https://cambridge-intelligence.com/how-webgl-powers-ux/), [2](https://www.educba.com/webgl-vs-canvas/)

WebGL - это кроссплатформенный компонент рендеринга для превосходной визуализации графики в браузере.

WebGL - это версия OpenGL, который представляет собой 3D-движок. Он помогает своему пользователю выполнять 3D-манипуляции в веб-браузерах. 

Основная отличительная черта WebGL - производительность. WebGL увеличивает скорость работы на счет двух аспектов:
* оптимизация вычислений;
* полное задействование ресурсов (WebGL делает это, перекладывая большую часть тяжелой работы на GPU)

Сравнение canvas и WebGL:
* canvas - популярный универсальный API, в основном ориентирован на 2D-графику. 
* canvas поддерживается широким спектром устаревших браузеров, прост в использовании и поставляется со многими функциями из коробки
* хотя это неплохой универсальный инструмент, он не такой гибкий и работает не так хорошо, как WebGL. 
* WebGL позволяет добиться лучшей частоты кадров.
* canvas - это предшественник WebGL; WebGL появился в результате экспериментов по 3D визуализации на canvas;
* canvas в основном используется для 2D изображений; WebGL - для 3D, но может работать и для 2D. 
* canvas и WebGL - оба являются JS API; 
* Canvas API намного проще в освоении и понимании, если пользователь хочет понимать и писать код с начального уровня (или с самого начала). Можно сказать, что для понимания canvas требуется минимальное понимание математики, в то время как WebGL нуждается в человеке, который хорошо разбирается в математике
* при использовании готовых библиотек особой разницы с точки зрения сложности уже нет.

[Пример работы с canvas и WebGL](assets/canvas_webgl.html)

## Работа с WebGL в Plotly

Существует 2 альтернативных решения для рендеринга изображений веб-браузерами:
1. SVG API: векторные изображения
2. Canvas  API: растровые изображения, могут использовать GPU для ускорения за счет использования WebGL.

Каждый тип трейсов в plotly в основном отображается либо в формате SVG, либо в WebGL, хотя трейсы на базе WebGL также используют некоторые элементы SVG.

Следующие типы трейсов используют (возможно, частично) WebGL:
* scattergl, scatterpolargl, heatmapgl - ускоренные версии "обычных" трейсов;
* splom, parcoords - высокопроизводительные многомерные трейсы;
* scatter3d, surface, mesh3d, .. - 3D трейсы

Ограничения:
* необходимость в наличии GPU
* WebGL рисует сетку из пикселей, а не сами фигуры, т.е. могут быть проблемы с качеством
* Текст рендерится по-разному для SVG и WebGL;
* браузеры ограничивают кол-во WebGL контекстов на странице; в plotly, возможно, не получится отобразить более 8 рисунков с использованием WebGL на одной странице одновременно;
* ограничения на размер фигуры.
* некоторые различия в работе ускоренных и неускоренных версий

## Примеры 

Первый вариант - отрисовать без использования WebGL. 
Результат:
* все объекты - векторные, находятся в DOM в виде SVG-тэгов;
* можно поменять стиль отдельного элемента;
* тэга canvas на полотне нет.

![PX без WebGL](assets/px_svg.png)

In [1]:
import plotly.express as px
import numpy as np
import pandas as pd

N = 100
df = pd.DataFrame(
    {
        "x": np.random.randn(N),
        "y": np.random.randn(N),
    }
)
fig = px.scatter(df, x="x", y="y", render_mode='svg')
fig.show(renderer='browser')

Для использования WebGL можно передать аргумент `render_mode="webgl"`.

Режим рендеринга по умолчанию - "auto", и в этом случае Plotly Express автоматически установит `render_mode="webgl"`, если длина входных данных превышает 1000 строк. Если ускорение WebGL в этом случае нежелательно, `render_mode` может быть принудительно изменен на "svg" для векторизованного, более медленного, рендеринга.

Результат:
* все объекты находятся в растровом виде на canvas в контейнере `.gl-container`
* векторных объектов (`.cartesianlayer.[...].plot`) - нет
* изменить свойства точек при помощи css не получится

![PX c WebGL](assets/px_webgl.png)

In [3]:
fig = px.scatter(df, x="x", y="y", render_mode='webgl')
fig.show(renderer='browser')

In [5]:
# можно явно создать фигуру и использовать ускоренные трейсы
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Scattergl(
        x = df['x'],
        y = df['y'],
        mode = 'markers'
    )
)

fig.show(renderer='browser')